# 迭代与推导

## 一、迭代协议：
- 可迭代对象(iterable)：
    - 迭代的被调对象，有\_\_iter\_\_方法，被iter函数调用
    - iter(iterable)的返回结果是迭代器对象
    - list, map, zip, range, dict, 文件, 生成器……
- 迭代器对象(iterator)：
    - 可迭代对象的返回结果，有\_\_next\_\_方法，被next函数调用
    - next(iterator)即可逐次返回迭代值
    - map, zip, 生成器, 文件……（许多既是迭代器对象，又是可迭代对象）


下面通过几个例子来区分可迭代对象和迭代器对象：

In [ ]:
# 举例：列表只是可迭代对象，不是迭代器对象
L = [1, 2, 3]
print(iter(L) is L)
next(L) # 报错，因为list不是迭代器对象，而是可迭代对象，需要用iter()来返回一个迭代器对象

In [ ]:
I_L = iter(L)
next(I_L) # 正确

In [ ]:
# 举例：文件既是可迭代对象，又是迭代器对象
file = open('data.txt')
print(iter(file) is file) # 确认文件的“双重身份”
print(next(file))
print(next(file))
print(next(file))
print(next(file))   # 迭代至末尾，会抛出StopIteration异常

In [ ]:

# 举例：range函数
r = range(5)
print(iter(r) is r) # range产生的是可迭代对象，而不是迭代器对象
I_r = iter(r)
print(next(I_r))

In [ ]:
# 举例：enumerate函数
e = enumerate('abcdef')
print(iter(e) is e) # enumerate产生的是可迭代对象，而不是迭代器对象
print(next(e))
print(next(e))

然而，在实际应用中，以for循环为代表的自动迭代可以自动生成迭代器对象，这极大地减轻了我们的负担：

In [ ]:
# 举例：列表，range的自动迭代
L = [1, 2, 3, 4]
for x in L: # L作为可迭代对象，在自动迭代（此处由for触发）中自动生成迭代器对象
    print(x)
print('\n')
for x in range(4):
    print(x) # range(4)作为可迭代对象，在自动迭代（此处由for触发）中自动生成迭代器对象 

在python中，字典是比较特殊的可迭代对象：
- 其自身可生成返回keys的可迭代对象
- 其keys, values, items方法均返回可迭代对象
- 但以上四者都不是迭代器

In [ ]:
d = dict(a=1, b=2, c=3)
print(iter(d) is d) # 字典只是可迭代对象，不是迭代器对象
I_self = iter(d)    # 仅以自身作为可迭代对象，生成的是返回keys的迭代器
print('\n自身：', d)
print('迭代自身：', [next(I_self), next(I_self)])   
I_keys = iter(d.keys()) # 仅以dict_keys作为可迭代对象
print('\n键：', d.keys())
print('迭代键：', [next(I_keys), next(I_keys)])   
I_values = iter(d.values()) # 仅以dict_values作为可迭代对象
print('\n值：', d.values())
print('迭代值：', [next(I_values), next(I_values)]) 
I_items = iter(d.items()) # 仅以dict_items作为可迭代对象
print('\n项：', d.items())
print('迭代项：', [next(I_items), next(I_items)])   


实际上，python中有无数的内置函数或对象方法是以可迭代对象作为参数的，下面仅是冰山一角，但足可以体现迭代协议在python中的广泛应用：

In [ ]:
# 举例：部分接纳可迭代对象的工具
# zip内置函数：接纳两个可迭代对象，返回可迭代对象
z = zip(open('data.txt'), [1, 2, 3])
print('\nzip:') 
print(z)
print(list(z))
# map内置函数：接纳一个函数（可自定义）与一个可迭代对象，返回一个可迭代对象
m = map(str.upper, open('data.txt'))    # 在这里，map接纳了python内置类型字符串(str)的方法upper作为第一个参数
print('\nmap:')
print(m)
print(list(m))
# sorted内置函数：返回排序后的列表
print('\nsorted:')
print(sorted(open('data.txt')))
# 内置类型字符串(str)中的join方法：返回拼接后的字符串
print('\nstr.join:')
print('......'.join(open('data.txt')))  # print之后会有换行，这是因为原字符串中有\n

不同内置函数迭代器的差别：是否支持在一个可迭代对象上建立多个“独立”的迭代器（即迭代器对象）
- range：支持
- map, zip, filter等：不支持
    - 对于只有单个迭代器的情形，一般意味着对象的迭代器就是其自身（即既是可迭代对象，又是迭代器对象）

请看以下例子：

In [ ]:
# range：支持多个迭代器
r = range(10)
I1 = iter(r)
I2 = iter(r)
[next(I1), next(I1), next(I1), next(I1)]    # 迭代I1
print('I1迭代至：',next(I1)) # I1已经迭代到比较后面
print('I2迭代至：',next(I2)) # I2还在初始位置

In [ ]:
# map：不支持多个迭代器
m = map(abs, [-i for i in range(10)])
I1 = iter(m)
I2 = iter(m)
[next(I1), next(I1), next(I1), next(I1)]    # 迭代I1
print('I1迭代至：',next(I1)) # I1迭代到比较后面
print('I2迭代至：',next(I2)) # I2不是停留在初始位置，而是与I1同步！
print(I1 is I2) # 显示为真：只有一个迭代器

## 二、推导
- 最主要的迭代协议上下文之一
- 较for循环，速度更快，推荐使用

In [ ]:
# 举例：最简单的列表推导
L = [1, 2, 3, 4, 5]
print([x**2 for x in L])
print(([x**2 for x in L if x % 2])  # 添加筛选分句if

In [ ]:
# 举例：文件中的列表推导
print([line for line in open('data.txt')])  
print([line.rstrip() for line in open('data.txt')]) # 可以在推导式中，对迭代结果进行进一步加工